In [1]:
import math
import numpy as np
import z3
from z3 import *

def car(real, imag):
    return complex(real,imag)

def pol(rho, phi):
    return (rho, phi)

def car2pol(car):
    x, y = car.real, car.imag
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    phi = phi / 2 / math.pi * 360
    return(rho, phi)

def pol2car(rho, phi):
    phi = phi / 360 * 2 * math.pi
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return complex(x, y)

In [45]:
def checkcar(var):
    if isinstance(var, tuple):
        print('converting {var} into carthesian to calculate with')
        return pol2car(*var)
    return var

def checkpol(var):
    if isinstance(var, complex):
        print('converting {var} into polar to calculate with')
        return car2pol(var)
    return var

def parallel(x1,x2):
    return (x1*x2) / (x1 + x2)

def current(voltage, resistance):
    voltage = checkcar(voltage)
    cur = voltage / resistance
    print('current in polar notation', car2pol(cur))
    return cur

def voltage(current, resistance):
    current = checkcar(current)
    vol = current * resistance
    print('voltage in polar notation', car2pol(vol))
    return vol

def deg2rad(deg):
    return deg / 360 * 2 * math.pi
def Q(voltage, current):
    voltage = checkpol(voltage)
    current = checkpol(current)
    print(f'  {voltage[0]} * {current[0]} * sin({voltage[1]} - {current[1]}) WARNING, make sure to multiply with -1 if the current enters at -')
    return voltage[0] * current[0] * math.sin(deg2rad(voltage[1] - current[1]))
    
def P(voltage, current):
    voltage = checkpol(voltage)
    current = checkpol(current)
    print(f'  {voltage[0]} * {current[0]} * cos({voltage[1]} - {current[1]}) WARNING, make sure to multiply with -1 if the current enters at -')
    return voltage[0] * current[0] * math.cos(deg2rad(voltage[1] - current[1]))

def S(P, Q):
    return math.sqrt(P**2 + Q**2)



In [3]:
def capacitor_resistance(capacity, frequency=50):
    return -1 / (2*math.pi * frequency) / capacity

def capacitor_capacity(resistance, frequency=50):
    return 1 / (resistance * 2*math.pi * frequency)
    
def inductor_resistance(capacity, frequency=50):
    return (2*math.pi * frequency) * capacity

def inductor_capacity(resistance, frequency=50):
    return resistance / (2*math.pi*frequency)

In [4]:
def car2complexexpr(cart):
    return ComplexExpr(cart.real, cart.imag)

def pol2complexexpr(size, angle):
    return car2complexexpr(pol2car(size, angle))

def _to_complex(a):
    if isinstance(a, ComplexExpr):
        return a
    else:
        return ComplexExpr(a, RealVal(0))

def _is_zero(a):
    return (isinstance(a, int) and a == 0) or (is_rational_value(a) and a.numerator_as_long() == 0)

class ComplexExpr:
    def __init__(self, r, i):
        self.r = r
        self.i = i

    def __add__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r + other.r, self.i + other.i)

    def __radd__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r + self.r, other.i + self.i)

    def __sub__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r - other.r, self.i - other.i)

    def __rsub__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r - self.r, other.i - self.i)

    def __mul__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r*other.r - self.i*other.i, self.r*other.i + self.i*other.r)

    def __mul__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r*self.r - other.i*self.i, other.i*self.r + other.r*self.i)

    def inv(self):
        den = self.r*self.r + self.i*self.i
        return ComplexExpr(self.r/den, -self.i/den)

    def __div__(self, other):
        inv_other = _to_complex(other).inv()
        return self.__mul__(inv_other)

    def __rdiv__(self, other):
        other = _to_complex(other)
        return self.inv().__mul__(other)

    def __eq__(self, other):
        other = _to_complex(other)
        return And(self.r == other.r, self.i == other.i)

    def __neq__(self, other):
        return Not(self.__eq__(other))

    def simplify(self):
        return ComplexExpr(simplify(self.r), simplify(self.i))

    def repr_i(self):
        if is_rational_value(self.i):
            return "%s*I" % self.i
        else:
            return "(%s)*I" % str(self.i)

    def __repr__(self):
        if _is_zero(self.i):
            return str(self.r)
        elif _is_zero(self.r):
            return self.repr_i()
        else:
            return "%s + %s" % (self.r, self.repr_i())

def Complex(a):
    return ComplexExpr(Real('%s.r' % a), Real('%s.i' % a))
# I = ComplexExpr(RealVal(0), RealVal(1))

def evaluate_cexpr(m, e):
    return ComplexExpr(m[e.r], m[e.i])

def get_ans(m, var):
    r = evaluate_cexpr(m, var).r
    i = evaluate_cexpr(m, var).i
    real = float(r.numerator_as_long())/float(r.denominator_as_long())
    imag = float(i.numerator_as_long())/float(i.denominator_as_long())
    return complex(real, imag)

In [5]:
indres = inductor_resistance(20/1000)
capres = capacitor_resistance(0.53/1000)


In [6]:
indres = ComplexExpr(0,indres)
capres = ComplexExpr(0,capres)

In [7]:
capres

(-6.005846909128126)*I

In [8]:
# template for AC nets
I1 = Complex("x")
I2 = Complex("y")
I3 = Complex("z")
s = Tactic('qfnra-nlsat').solver()

s.add(pol2complexexpr(100,0) - I1 * indres - I3 * capres == 0)
s.add(I3 * capres + ComplexExpr(10,0) * I2 == 0)
s.add(I1 + I2 - I3 == 0)



# s.add(ComplexExpr(cur.real, cur.imag) - I1 * ComplexExpr(8,4) - I2*ComplexExpr(0,-6) == 0)
# s.add(I2 * ComplexExpr(0,-6) - ComplexExpr(cur2.real, cur2.imag) == 0)
# s.add(I1 + I3 - I2 == 0)

# s.add(100-I1*ComplexExpr(0,6.28)-I3*ComplexExpr(0,-6)==0)
# s.add(I3*ComplexExpr(0,-6)+I2*10 == 0)
# s.add(I1 + I2 -I3 == 0)
s.check()
m = s.model()

for var in [I1,I2, I3]:
    print(car2pol(get_ans(m,var)))
    

(30.828871213696655, -35.191760224510716)
(15.872684148367618, 85.79662254309166)
(26.42871919402932, -4.203377456908344)


In [9]:
I1 = (30.828871213696655, -35.191760224510716)
I2 = (15.872684148367618, 85.79662254309166)
I3 = (26.42871919402932, -4.203377456908344)
-P((100,0), I1), -Q((100,0), I1), S(-P((100,0), I1), -Q((100,0), I1))
car2pol(complex(*I1))

100 * 30.828871213696655 * cos(0 - -35.191760224510716)
WARNING, make sure to multiply with - if the current enters at - of source
100 * 30.828871213696655 * sin(0 - -35.191760224510716)
WARNING, make sure to multiply with - if the current enters at - of source
100 * 30.828871213696655 * cos(0 - -35.191760224510716)
WARNING, make sure to multiply with - if the current enters at - of source
100 * 30.828871213696655 * sin(0 - -35.191760224510716)
WARNING, make sure to multiply with - if the current enters at - of source


(46.785460220138354, -48.78082103360182)

In [10]:
# template for DC nets
I1 = Real('x')
I2 = Real('y')
I3 = Real('z')
I4 = Real('r')
I5 = Real('t')

s = Tactic('qfnra-nlsat').solver()
s.add(-10*I1-100-5*I2==0)
s.add(5*I2 + 100 -20*I4 == 0)
s.add(20*I4 + 40 == 0)

s.add(I1 - I2 - I3 == 0)
s.add(I3 - I4 - I5 == 0)
    
s.check()
m = s.model()

m.eval(I1), m.eval(I2), m.eval(I3),m.eval(I4),m.eval(I5)

(4, -28, 32, -2, 34)

In [11]:
inductor_capacity(10)

0.03183098861837907

In [12]:
capacitor_capacity(5)

0.0006366197723675814

In [13]:
# template for DC nets
R = Real('x')
I2 = Real('y')
I3 = Real('z')
I4 = Real('r')
I5 = Real('t')

s = Tactic('qfnra-nlsat').solver()
s.add(40 == (60 * (16+R)) / (60 + (16+R)))
    
s.check()
m = s.model()

m.eval(R)

-76

In [14]:
Q((100,0),(17,-72.1))

100 * 17 * sin(0 - -72.1)
WARNING, make sure to multiply with - if the current enters at - of source


1617.7104865950448

In [15]:
Q((50,30),(6.5,36.5))

50 * 6.5 * sin(30 - 36.5)
WARNING, make sure to multiply with - if the current enters at - of source


-36.79104447456968

In [16]:
car2pol(pol2car((16.1),-49.5)*-5j)

(80.5, -139.50000000000003)

In [17]:
car2pol((pol2car((16.1),-49.5)*-5j - pol2car(50,30))/20)

(6.499125326897812, -143.51973203948765)

In [72]:
from collections import namedtuple
Power = namedtuple('Power', 'Watt VAR')

def power(current, resistance):
    if isinstance(resistance, complex):
        return Power(power(current, resistance.real)[0], 
                     power(current, resistance.imag)[0])
    else:
        return Power(car2pol(current ** 2 * resistance)[0],0)


def trans(primaryvoltage, primaryresistance, secondaryresistance, ratio):
    # bring everything to left side
    primaryvoltage = checkcar(primaryvoltage)
    res = primaryresistance + ratio ** 2 * secondaryresistance
    Iprim = current(primaryvoltage, res)
    Isec = Iprim * ratio
    
    Uprim = checkcar(primaryvoltage) - primaryresistance * Iprim
    Usec  = Uprim / ratio
    print(f'Outcomes for transformer: \nIp {car2pol(Iprim)}\nIs {car2pol(Isec)}\nUp {car2pol(Uprim)}\nUs {car2pol(Usec)}')
    print(f'Outcomes for powers on primary side\nPsource {P(primaryvoltage, Iprim)}, Qsource{Q(primaryvoltage, Iprim)}\n')
    print(f'Outcomes for resistances on primary side\nP {power(Iprim, primaryresistance)}\n')
    print(f'Outcomes for resistances on secondary side\nP {power(Isec, secondaryresistance)}\n')

trans((100,0), 2, (100-200j), 0.1)

converting {var} into carthesian to calculate with
current in polar notation (27.735009811261456, 33.6900675259798)
Outcomes for transformer: 
Ip (27.735009811261456, 33.6900675259798)
Is (2.7735009811261455, 33.6900675259798)
Up (62.017367294604234, -29.744881296942225)
Us (620.1736729460423, -29.74488129694223)
converting {var} into polar to calculate with
converting {var} into polar to calculate with
  100.0 * 27.735009811261456 * cos(0.0 - 33.6900675259798) WARNING, make sure to multiply with -1 if the current enters at -
converting {var} into polar to calculate with
converting {var} into polar to calculate with
  100.0 * 27.735009811261456 * sin(0.0 - 33.6900675259798) WARNING, make sure to multiply with -1 if the current enters at -
Outcomes for powers on primary side
Psource 2307.6923076923076, Qsource-1538.4615384615388

Outcomes for resistances on primary side
P Power(Watt=1538.4615384615383, VAR=0)

Outcomes for resistances on secondary side
P Power(Watt=769.2307692307692, VA

In [73]:
trans((10000,0), 60j, 20, 2)


converting {var} into carthesian to calculate with
current in polar notation (100.0, -36.86989764584402)
Outcomes for transformer: 
Ip (100.0, -36.86989764584402)
Is (200.0, -36.86989764584402)
Up (8000.0, -36.86989764584402)
Us (4000.0, -36.86989764584402)
converting {var} into polar to calculate with
converting {var} into polar to calculate with
  10000.0 * 100.0 * cos(0.0 - -36.86989764584402) WARNING, make sure to multiply with -1 if the current enters at -
converting {var} into polar to calculate with
converting {var} into polar to calculate with
  10000.0 * 100.0 * sin(0.0 - -36.86989764584402) WARNING, make sure to multiply with -1 if the current enters at -
Outcomes for powers on primary side
Psource 800000.0, Qsource600000.0

Outcomes for resistances on primary side
P Power(Watt=0.0, VAR=600000.0)

Outcomes for resistances on secondary side
P Power(Watt=800000.0, VAR=0)



In [74]:
trans((100,0), 20,5, 4)

converting {var} into carthesian to calculate with
current in polar notation (1.0, 0.0)
Outcomes for transformer: 
Ip (1.0, 0.0)
Is (4.0, 0.0)
Up (80.0, 0.0)
Us (20.0, 0.0)
converting {var} into polar to calculate with
converting {var} into polar to calculate with
  100.0 * 1.0 * cos(0.0 - 0.0) WARNING, make sure to multiply with -1 if the current enters at -
converting {var} into polar to calculate with
converting {var} into polar to calculate with
  100.0 * 1.0 * sin(0.0 - 0.0) WARNING, make sure to multiply with -1 if the current enters at -
Outcomes for powers on primary side
Psource 100.0, Qsource0.0

Outcomes for resistances on primary side
P Power(Watt=20.0, VAR=0)

Outcomes for resistances on secondary side
P Power(Watt=80.0, VAR=0)

